# Introduction

   **Chennai** also known as **Madras** is the capital of the **Indian state of Tamil Nadu**. Chennai is among the **most-visited Indian cities by foreign tourists**. Tourism-guide publisher **Lonely Planet** named Chennai as **one of the top ten cities in the world to visit in 2015**. Chennai was ranked the **43rd-most visited city in the world** for the year 2015. The **Quality of Living Survey** rated Chennai as the **safest city in India**. Chennai attracts 45 percent of health tourists visiting India, and 30 to 40 percent of domestic health tourists. **National Geographic** mentioned Chennai as the **only South Asian city** to feature in its 2015 **"Top 10 food cities"** list. Chennai was also named the **ninth-best cosmopolitan city in the world** by Lonely Planet. In October 2017, Chennai was added to the **UNESCO Creative Cities Network (UCCN) list for its rich musical tradition**.

### Business Problem

In recent days, the **tourism of this beautiful city has decreased**. Tourism not only adds to the country's economy but is a core part of income. How do we increase tourism and bring back the popularity of the city ? In-order to achieve this, we need the **tourists to have a good experience and moreover a personalized one**. We can achieve this with the help of data science.

### Target Audience

With all this tourist attraction taken into account, a system that can find a place suitable for the tourist to vist during their stay will be helpful. Our goal is to **identify places based on their rating & pricing** and make it visible to the tourists in-order for them **to choose a place to visit based on their budget and based on the venue's rating**.
   
   We will be clustering the places to visit based on the data we recieve from the **Foursquare API** & **Zomato API**.

# Data

Our objective is to search within a **5km radius of Chennai** located at the **Latitude : 13.0827° N** & **Longitude : 80.2707° E**. Using the FoursquareAPI & ZomatoAPI the data mentioned below is retrieved. Based on the Venue Name, Latitude & Longitude obtained from the FoursquareAPI we request for data from the ZomatoAPI. We have collected a total of **119 Venues within a 5km radius** of the geographical co-ordinates of Chennai. Since we are using two datasets each from different API, there maybe some noise. In-order to clean this data, we are eliminating venues with latitude & longitude values more than 0.0004. On cleaning this data we are left with **74 venues** to obtain a working model.

The following data has been collected from the **FoursquareAPI** :
* Venue Name
* Category
* Latitude
* Longitude

The following data has been collected from the **ZomatoAPI** :
* Average Price for Two People
* Price Range
* Rating
* Address

### Example of Data

In [2]:
import requests
from pandas.io.json import json_normalize
import pandas as pd

client_id = 'SPJ5ZZH2JCBKLK3Y5HF51JBNPK5GGGPI2J43MDND3Z3MFD2L'
client_secret = 'Y1J35FIYYFC1LUW3ATM0TVBTB1E3TVY4ALG2F2FZXSXYNCFB'
version = '20180605'
chennai_latitude = 13.0827
chennai_longitude = 80.2707
radius = 5000 #5KM
limit = 3
offset = 3
fs_venues = pd.DataFrame(columns = ['name', 'categories', 'lat', 'lng'])

def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

while(True):
    url = ('https://api.foursquare.com/v2/venues/explore?client_id={}'
           '&client_secret={}&v={}&ll={},{}&radius={}&limit={}&offset={}').format(client_id, 
                                                                        client_secret, 
                                                                        version, 
                                                                        chennai_latitude, 
                                                                        chennai_longitude, 
                                                                        radius,
                                                                        limit,
                                                                        offset)
    result = requests.get(url).json()
    venues_fetched = len(result['response']['groups'][0]['items'])
    venues = result['response']['groups'][0]['items']
    venues = json_normalize(venues)

    # Filter the columns
    filtered_columns = ['venue.name', 'venue.categories', 'venue.location.lat', 'venue.location.lng']
    venues = venues.loc[:, filtered_columns]
    
    # Filter the category for each row
    venues['venue.categories'] = venues.apply(get_category_type, axis = 1)
    
    # Clean all column names
    venues.columns = [col.split(".")[-1] for col in venues.columns]
    fs_venues = pd.concat([fs_venues, venues], axis = 0, sort = False)
    
    if (venues_fetched < 100):
        break
    else:
        offset = offset + 100

fs_venues = fs_venues.reset_index(drop = True)
fs_venues.rename(columns = {'name':'Venue Name','categories':'Category','lat':'Latitude','lng':'Longitude'})

,Venue Name,Category,Latitude,Longitude
0,Seena bhai tiffen centre,Restaurant,13.089670,80.278455
1,Escape Cinemas,Multiplex,13.058746,80.264170
2,Shree Mithai,Indian Restaurant,13.072118,80.247865
